In [12]:
import random

# Function to perform bitwise XOR
def bitwise_xor(binary_str1, binary_str2):
    return ''.join(str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(binary_str1, binary_str2))

# Function to perform left rotation
def left_rotation(X, Y):
    wt_y = Y.count('1')  # Hamming weight of Y
    L = len(X)  # Calculate L based on the length of X
    wt_y_mod_L = wt_y % L
    return X[wt_y_mod_L:] + X[:wt_y_mod_L]

# Function to perform Right rotation
def right_rotation(X, Y):
    wt_y = Y.count('1')  # Hamming weight of Y
    L = len(X)  # Calculate L based on the length of X
    wt_y_mod_L = wt_y % L
    return X[-wt_y_mod_L:] + X[:-wt_y_mod_L]

# Function to generate a random binary string of a given length
def generate_random_binary_string(length):
    return ''.join(str(random.randint(0, 1)) for _ in range(length))

# Function to simulate the permutation function
def permutation(X, Y):
    assert len(X) == len(Y), "Bit strings X and Y must have equal length"
    m = Y.count('1')  # Hamming weight of Y
    km_indices = [i for i in range(len(Y)) if Y[i] == '1']
    kn_indices = [i for i in range(len(Y)) if Y[i] == '0']
    # Construct the permutation Z 
    Z = ''.join([X[k] for k in km_indices[::-1]]) + ''.join([X[k] for k in kn_indices[::-1]])
    return Z

def inverse_permutation(Z, Y):
    assert len(Z) == len(Y), "Bit strings Z and Y must have equal length"
    m = Y.count('1')  # Hamming weight of Y
    km_indices = [i for i in range(len(Y)) if Y[i] == '1']
    kn_indices = [i for i in range(len(Y)) if Y[i] == '0']
    # Split Z into km and kn parts
    Z_km = Z[:m][::-1]
    Z_kn = Z[m:][::-1]
    # Reconstruct X using Z, km_indices, and kn_indices
    X = ['0'] * len(Y)
    for i in range(m):
        X[km_indices[i]] = Z_km[i]
    for i in range(len(Y) - m):
        X[kn_indices[i]] = Z_kn[i]
    return ''.join(X)

In [13]:
# Initialization Phase
l = 128 # The length of the values
# Step 1: Generate a l-bit random value for IDSx
IDSx = generate_random_binary_string(l)

# Step 2: Generate a l-bit random value for IDn
IDn = generate_random_binary_string(l)

# Step 3: Generate a l-bit random value for KR (temporary shared secret)
KR = generate_random_binary_string(l)

# Step 4: Generate a l-bit random value for KT (temporary shared secret)
KT = generate_random_binary_string(l)

# Step 5: Use the permutation function to create index data
index_data = {"fx(IDS1, ID1)": permutation(IDSx, IDn)}

# Define l-bit binary values for K1, K2, and K3 
K1 = generate_random_binary_string(l)
K2 = generate_random_binary_string(l)
K3 = generate_random_binary_string(l)

# Define a l-bit binary value for IDo (Identifier of the old owner)
IDo = generate_random_binary_string(l)

# Print the generated values
print("Initialization Phase:")
print("Generated IDo:", IDo)
print("Generated K1:", K1)
print("Generated K2:", K2)
print("Generated K3:", K3)
print("Generated IDSx:", IDSx)
print("Generated IDn:", IDn)
print("Generated KR:", KR)
print("Generated KT:", KT)
print("Index Data:", index_data)

Initialization Phase:
Generated IDo: 01111110111110001101011111001011101100110011000011100100100101110110100100010101001110110101110101010011010111111101000001011001
Generated K1: 00001110101001000000011101001010000101001010001111011010101000000011011010100011001101001100111111001110010001101011101110111000
Generated K2: 00010100011011111110000100111011010111011011001110010110110010000011011111000111001100111100101101011010110011111001000110011101
Generated K3: 00000101001110110001100001110000100111100101001111100111000110101010000101010011101001011101010111101000000100110011001111111001
Generated IDSx: 11111110000001011010010101100111101100110000110110000110011001111111110001001001011001111001000100011111010111000100111111000000
Generated IDn: 01001000110010011110001100001000100011110010010011110111001110100001110111101010100111111101000101000100100010110111000101110111
Generated KR: 010101001101010010000011101110101101110000100000100110101010110011010000010010011101010100011101110000

In [14]:
# Step 1 & 2 (Based on Figure)
# RN

# Generate a random l-bit value for r1 using PRNG
r1 = generate_random_binary_string(l)

# Calculate A = r1 ⊕ KT
A = bitwise_xor(r1, KT)

# Send "hello" and A to the tag
message_to_tag = "hello"
#message_to_tag_binary = ''.join(format(ord(char), '08b') for char in message_to_tag)  # Convert "hello" to binary

# Concatenate the binary message and A
message_to_tag = (message_to_tag, A)

# Print the generated values and the message to the tag
#print("Generated r1:", r1)
#print("Generated A:", A)
print("Message to tag:", message_to_tag)

Message to tag: ('hello', '10000000101011000100100101000100010110001001111100101010110011001110110100001111100011010111011010000101100111011100111000111110')


In [15]:
# Step 3 & 4
# Tag

# Extract r'1 from A by performing a bitwise XOR with KT
r1_prime = bitwise_xor(A, KT)
#print("r1_prime:", r1_prime)

# Calculate B' using the permutation function with inputs (r'1 ⊕ IDSx, KT)
r1_prime_xor_IDSx = bitwise_xor(r1_prime, IDSx)
B_prime = permutation(r1_prime_xor_IDSx, KT)

# Send IDSx and B' to the new owner (RN)
print("IDSx (to be sent to the new owner):", IDSx)
print("Generated B' (to be sent to the new owner):", B_prime)

IDSx (to be sent to the new owner): 11111110000001011010010101100111101100110000110110000110011001111111110001001001011001111001000100011111010111000100111111000000
Generated B' (to be sent to the new owner): 10001101000001001010100110101101110110101000100001011001010101000111110000100001010100101101100011011010000011110100000111101110


In [16]:
# Step 5 & 6
# RN

# Calculate B using the permutation function with inputs (r1 ⊕ IDSx, KT)
r1_xor_IDSx = bitwise_xor(r1, IDSx)
B = permutation(r1_xor_IDSx, KT)

# Check if B equals B'
if B == B_prime:
    # Case 1: B equals B'
    # Generate a new random number r2 using PRNG
    r2 = generate_random_binary_string(l)
    #print("r2:", r2)
    
    # Calculate C = fx(r2 ⊕ IDSx, KR)
    r2_xor_IDSx = bitwise_xor(r2, IDSx)
    C = permutation(r2_xor_IDSx, KR)

    # Calculate D = Rot(r2, fx(IDn, KR))
    D = left_rotation(r2, permutation(IDn, KR))

    # Calculate fx(IDSx, IDn)
    fx_IDSx_IDn = permutation(IDSx, IDn)

    # Send IDSx, C, D, and fx(IDSx, IDn) to the old owner (RC)
    print("Generated IDSx (to be sent to the old owner):", IDSx)
    print("Generated C (to be sent to the old owner):", C)
    print("Generated D (to be sent to the old owner):", D)
    print("Generated fx(IDSx, IDn) (to be sent to the old owner):", fx_IDSx_IDn)

else:
    # Case 2: B is not equal to B', terminate the protocol
    print("B is not equal to B'. Protocol terminated.")


Generated IDSx (to be sent to the old owner): 11111110000001011010010101100111101100110000110110000110011001111111110001001001011001111001000100011111010111000100111111000000
Generated C (to be sent to the old owner): 11101000010001111100010110010110111011001001001000001110010001000110111100101101100001010001011111101011010101100100100010001110
Generated D (to be sent to the old owner): 11001001001100001100000001011110000110010010110110111100000010111000000100001111011010000110111000111011001101101011011100100001
Generated fx(IDSx, IDn) (to be sent to the old owner): 00000110010010101101111000010100111100101100011011001010101100011011110110111110000001110001111110010100011011101101000100011111


In [17]:
# Step 7 & 8
# RC

# Search for the corresponding index content in the index data table

if fx_IDSx_IDn in index_data.values():
    # Case 1: The result matches, ownership transfer request is legal

    # Extract r2 from C using fx, IDSx, and KR
    r2_xor_IDSx = inverse_permutation(C, KR)
    r2_prime = bitwise_xor(r2_xor_IDSx, IDSx)
    #print("r2_extracted:", r2_prime)
    
    # Recalculate D' = Rot(r'2, fx(IDn, KR))
    D_prime = left_rotation(r2_prime, permutation(IDn, KR))

    # Check if D' equals D
    if D_prime == D:
        # Calculate random number r3
        r3 = generate_random_binary_string(l)
        # Calculate E = fx(r3, r2) ⊕ KR
        E = bitwise_xor(permutation(bitwise_xor(r3, r2_prime), KR), KR)

        # Calculate F = Rot(r2 ⊕ KR, fx(IDo, r3))
        F = left_rotation(bitwise_xor(r2_prime, KR), permutation(IDo, r3))

        # Calculate G = fx(K1 ⊕ K2, K2 ⊕ K3)
        G = permutation(bitwise_xor(K1, K2), bitwise_xor(K2, K3))

        # Calculate H = Rot(K2 ⊕ K3, IDn ⊕ KR)
        H = left_rotation(bitwise_xor(K2, K3), bitwise_xor(IDn, KR))

        # Send E, F, G, and H to the new owner (RN)
        print("Generated E (to be sent to the new owner):", E)
        print("Generated F (to be sent to the new owner):", F)
        print("Generated G (to be sent to the new owner):", G)
        print("Generated H (to be sent to the new owner):", H)
    else:
        # D' is not equal to D, terminate the protocol
        print("D' is not equal to D. Protocol terminated.")
else:
    # Case 2: The result does not match, terminate the protocol
    print("Result does not match. Ownership transfer request is not legal. Protocol terminated.")


Generated E (to be sent to the new owner): 00011110000011011011011101000000111000010011001101000000111011011110101000000001110001100100000001001000100011001010100011111010
Generated F (to be sent to the new owner): 11010001101101010011001011001100001001111110101011000100100101000101001100110111100011011100000110111011110000010011110000110100
Generated G (to be sent to the new owner): 11011001001010101001100100000000100001000101010010001110010110000001001000100000010000001110000011011000001010001101111101101000
Generated H (to be sent to the new owner): 10100101001011010010100100101100001111010110010110111001010001001100100000100010101010011111001010010111100001111100000011100011


In [18]:
# Step 9 & 10
# RN

# Extract r'3 from E
E_xor_KR = bitwise_xor(E, KR)
r3_prime = inverse_permutation(E_xor_KR, r2)

# Calculate F' = Rot(r2 ⊕ KR, fx(IDo, r'3))
F_prime = left_rotation(bitwise_xor(r2, KR), permutation(IDo, r3_prime))

# Check if F' equals F
if F_prime == F:
    # Case 1: F is successfully verified
    #Extract K2 ⊕ K3 from H
    K2_xor_K3 = right_rotation(H, bitwise_xor(IDn, KR))
    
    # Generate a random number r4 using PRNG
    r4 = generate_random_binary_string(l)

    # Calculate I = fx(r4 ⊕ KT, K2 ⊕ K3)
    I = permutation(bitwise_xor(r4, KT), K2_xor_K3)

    # Calculate J = Rot(fx(r4, KT), r4 ⊕ IDSx)
    J = left_rotation(permutation(r4, KT), bitwise_xor(r4, IDSx))

    # Calculate G ⊕ I
    G_xor_I = bitwise_xor(G, I)
    
    # Send G ⊕ I and J to the tag
    print("Generated G ⊕ I (to be sent to the tag):", G_xor_I)
    print("Generated J (to be sent to the tag):", J)
else:
    # Case 2: F is not successfully verified, abort the protocol
    print("F is not successfully verified. Protocol aborted.")


Generated G ⊕ I (to be sent to the tag): 01001010100010110100000101011010010010101101010001101111101101100100110110110110000101000010000110010100110110111000111100111110
Generated J (to be sent to the tag): 11111110000000111111000001101010111010000000101010000110001010100010011101110111000110011100111000100011100011101010001101100010


In [19]:
# Step 11 & 12
# Tag

#Extract r'4 from G ⊕ I
K1_xor_K2_xor_r4_xor_KT = inverse_permutation(G_xor_I, bitwise_xor(K2, K3))
K1_xor_K2 = bitwise_xor(K1, K2)
K1_xor_K2_xor_KT = bitwise_xor(K1_xor_K2, KT)
r4_prime = bitwise_xor(K1_xor_K2_xor_r4_xor_KT, K1_xor_K2_xor_KT)

# Calculate J' = Rot(fx(r4, KT), r4 ⊕ IDSx)
r4_xor_IDSx = bitwise_xor(r4_prime, IDSx)
fx_r4_KT = permutation(r4_prime, KT)
J_prime = left_rotation(fx_r4_KT, r4_xor_IDSx)

# Check if J' == J
if J_prime == J:
    # Tag deems the old owner authorized to transfer ownership
    print("Tag deems the old owner authorized to transfer ownership")
    # Calculate L = fx(r4 ⊕ r1, KT) ⊕ r4
    r4_xor_r1 = bitwise_xor(r4, r1)
    L = bitwise_xor(permutation(r4_xor_r1, KT), r4)

    # Send L to the new owner
    print("L (to be sent to the new owner):", L)
else:
    # Tag does not authorize ownership transfer
    print("Ownership transfer not authorized.")

Tag deems the old owner authorized to transfer ownership
L (to be sent to the new owner): 10001111010111101111111110110010011000000100000000101101011000101111001001000110110111111010010111010101001111100011101001100001


In [20]:
# Step 13 & 14
# RN

# Calculate L'
r4_xor_r1 = bitwise_xor(r4, r1)
L_prime = bitwise_xor(permutation(r4_xor_r1, KT), r4)

# Verify L and handle the cases
if L == L_prime:
    # Generate a random value for r5
    r5 = generate_random_binary_string(l)

    # Calculate M = fx(r4, KT) ⊕ r5
    fx_r4_KT = permutation(r4, KT)
    M = bitwise_xor(fx_r4_KT, r5)

    # Calculate N = Rot(r5, fx(r4, KT ⊕ IDSx))
    KT_xor_IDSx = bitwise_xor(KT, IDSx)
    fx_r4_KT_IDSx = permutation(r4, KT_xor_IDSx)
    N = left_rotation(r5, fx_r4_KT_IDSx)

    # Update pseudonym and shared keys
    IDSx_new = permutation(bitwise_xor(IDSx, r4), r5)
    #K1_xor_K2 = inverse_permutation(G, K2_xor_K3)
    K1_new = bitwise_xor(permutation(inverse_permutation(G, K2_xor_K3), r4), r5)
    K2_new = permutation(K2_xor_K3, r5)
    K3_new = permutation(bitwise_xor(KT, r4), r5)
 
    # Case 1: Send M, N to the tag. Update IDSx_new, K1_new, K2_new, K3_new, KT_new
    print("M (to be sent to the tag):", M)
    print("N (to be sent to the tag):", N)
    print("IDSx_new:", IDSx_new)
    print("K1_new:", K1_new)
    print("K2_new:", K2_new)
    print("K3_new:", K3_new)
    
else:
    # Case 2: abort the protocol
    print("Verification failed. Abort the protocol.")

M (to be sent to the tag): 11101101011110011111100001101001000111011100100001011101011110010111000100011100010010000110010010010110110110000111100010111011
N (to be sent to the tag): 01000010011111000100100101110010100111111000011000001110010001111011100011101011110100011110111000011010011111010111101001111011
IDSx_new: 01001011100111001000101010101110011111101110101111100100011001100010111111101110010111111101001111110011010100000101100100110000
K1_new: 10111101110010101000001101100100110100001110100100010100111011100001001011001010100011011001001111011001000100001010110100001001
K2_new: 01001101010001101110010000010111011110110000111110110011001101011100000100001001111010110011010101100010000010000011010111000010
K3_new: 11100110111010000100110001110011001100011001101000000111101001011101101001010111101001111010101000100000110111110100100000001001


In [21]:
# Step 15
# Tag

# Calculate r'5 from M
r5_prime = bitwise_xor(M, permutation(r4, KT))

# Calculate the local value of N
N_prime = left_rotation(r5_prime, permutation(r4, bitwise_xor(KT, IDSx)))

# Verify if the calculated N matches the received N
if N_prime == N:
    # Verification successful, update the pseudonym and shared secret
    IDSx_new = permutation(bitwise_xor(IDSx, r4), r5_prime)
    K1_new = bitwise_xor(permutation(bitwise_xor(K1, K2), r4), r5_prime)
    K2_new = permutation(bitwise_xor(K2, K3), r5_prime)
    K3_new = permutation(bitwise_xor(KT, r4), r5_prime)

    # Print the updated pseudonym and shared secret
    print("IDSx_new:", IDSx_new)
    print("K1_new:", K1_new)
    print("K2_new:", K2_new)
    print("K3_new:", K3_new)

else:
    # Verification failed
    print("Verification failed. Do not update pseudonym and shared secret.")

IDSx_new: 01001011100111001000101010101110011111101110101111100100011001100010111111101110010111111101001111110011010100000101100100110000
K1_new: 10111101110010101000001101100100110100001110100100010100111011100001001011001010100011011001001111011001000100001010110100001001
K2_new: 01001101010001101110010000010111011110110000111110110011001101011100000100001001111010110011010101100010000010000011010111000010
K3_new: 11100110111010000100110001110011001100011001101000000111101001011101101001010111101001111010101000100000110111110100100000001001


In [22]:
#Old Owner

if IDSx_new != IDSx:
    IDSx = IDSx_new
    K1 = K1_new
    K2 = K2_new
    K3 = K3_new
elif IDSx_new == IDSx:
    IDSx = IDSx
    K1 = K1
    K2 = K2
    K3 = K3
    
print("Updated IDSx:", IDSx)
print("Updated K1:", K1)
print("Updated K2:", K2)
print("Updated K3:", K3)


Updated IDSx: 01001011100111001000101010101110011111101110101111100100011001100010111111101110010111111101001111110011010100000101100100110000
Updated K1: 10111101110010101000001101100100110100001110100100010100111011100001001011001010100011011001001111011001000100001010110100001001
Updated K2: 01001101010001101110010000010111011110110000111110110011001101011100000100001001111010110011010101100010000010000011010111000010
Updated K3: 11100110111010000100110001110011001100011001101000000111101001011101101001010111101001111010101000100000110111110100100000001001
